In [ ]:
import json
from time import sleep

import seaborn as sns
import matplotlib.pyplot as plt
from pypfopt.efficient_frontier import EfficientFrontier, EfficientCVaR
from pypfopt import risk_models, expected_returns

from library.data import *

In [ ]:
# パラメータの設定
str_date = "2017-04-01"
end_date = "2022-03-31"
with open("assets.json", "r") as f:
    asset_list = json.load(f)

In [ ]:
# スクレイピング
df_list = []
for asset in asset_list:
    df = get_data(ISIN=asset["ISIN"], FUND=asset["FUND"])
    df_list.append(df)
    sleep(1)

df_join = None
for asset, df in zip(asset_list, df_list):
  df_join = join_data(df, df_join, asset["name"], str_date, end_date)

In [ ]:
# 時系列をプロット
plt.rcParams["font.family"] = 'Hiragino Sans'  
for asset in asset_list:
    plt.plot(df_join[asset["name"]], label=asset["name"])
plt.legend()

In [ ]:
# 相関係数を可視化
df_corr = df_join.corr()
sns.set(font='Hiragino Sans')
sns.heatmap(df_corr)

In [ ]:
# 効率的フロンティアの計算
mu = expected_returns.mean_historical_return(df_join)
S = risk_models.sample_cov(df_join)
ef = EfficientFrontier(mu, S)
ef.efficient_return(target_return=0.1)

In [ ]:
# CVaR最小化
returns = expected_returns.returns_from_prices(df_join)
ef_CVaR = EfficientCVaR(expected_returns=mu, returns=returns, beta=0.95)
ef_CVaR.efficient_return(target_return=0.1)